In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab1.ipynb")

In [ ]:
import numpy as np
from numpy import sin, cos, pi, sqrt, cross, dot, array, linspace, arange
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# import preamble
# import importlib
# importlib.reload(preamble)

from preamble import *

from scipy.integrate import quad

from IPython.display import HTML, display



In [ ]:
%%html

<style>
.shadow {

    /*Edit or add new attributes, change size, color, etc */
    width: 75%;
    box-shadow: 8px 8px 10px #444;
    border: 1px solid silver;

    /*For positioning in a jupyter notebook*/
    margin-top: 2em;
    position: relative;
    top: -25px
}
</style>

# Lab 1 - Space Cave Runner

Recently we have used vector-valued functions to model and study motion in space. Let's put it to use in a knock-off of a classic game<sup>1</sup>. We are operating a ship in space and must navigate it through a cave to get to the Green Zone. How is there a cave but no gravity? That's not important. 

![Unsuccessful Demo Run](bonk.gif)

<div class="alert alert-block alert-info shadow" style="min-height: 150px;">
<img src="asteroids-screenshot.jpeg" width="150px" style="float: left; padding: 5px;">
<sup>1</sup>Remember <b>Asteroids</b>? Perhaps not, but I had a version on my 386 and it was fantastic. Here's a <a href="https://games.aarp.org/games/atari-asteroids">version you can play</a> at the, uh, AARP.
</div>

Anyway, I want to talk about how that ship moves. We will ignore shooting and hyperjumping. Of course the ship has a location $(x,y)$ but it also can point in any direction. We'll call that angle $\alpha$ and measure it **clockwise** off of vertical. 

![Diagram showing angular displacement of ship](ship_alpha.png)

### Question 1

Find the **unit** vector $\mathbf u_\alpha$ that points in the direction the ship is facing as a function of $\alpha$.  

In [ ]:
def u_alpha(alpha):
    """Returns the unit vector alpha radians clockwise from vertical."""
    x = ...
    y = ...
    return np.array((x,y))

In [ ]:
grader.check("q1")

## States and Controls

Now we can draw the ship at any moment using just the 3-vector $\langle x, y, \alpha \rangle$, but if we want to figure out where its going, we need a little more.

The ship has a single thruster that pushes with constant force from the back and only has modes on or off. It can also rotate around a central point. Units are normalized so that its acceleration is $$\mathbf a(t) = \begin{cases} 
\mathbf u_\alpha & \text{ if thrust on} \\
\mathbf 0 & \text{ if thrust off}
\end{cases}$$


The ship's controls consist of exactly 3 buttons: 

<span style="padding: 4px; border-radius: 4px; border: 1px solid black; color: red">Thrust</span> pushes forward at $1 \frac{\text{unit}}{\text{s}^2}$

<span style="padding: 4px; border-radius: 4px; border: 1px solid black; color: green">Left</span> turns ship left at $2\pi$ radians per second

<span style="padding: 4px; border-radius: 4px; border: 1px solid black; color: blue">Right</span> turns ship right at $2\pi$ radians per second

So that looks like $2^3 = 8$ possible control states, but in fact, it's just 6, as pushing left and right together is the same as none. Thus we can model our control state with a vector $$\mathbf c(t) = \langle c_1, c_2 \rangle$$ where $c_1$ is only $0$ or $1$ whether the thrust is off or on, and $c_2$ is $-1$, $0$, $1$ whether the controls are left, center, or right, respectively.

So, if we know when which buttons are pressed, $\mathbf c(t)$, and we know where the ship starts, we can compute where it goes. To make that easier we define a **state vector** $$\mathbf s(t) = \langle x, y, \dot x, \dot y, \alpha \rangle$$ that includes the velocities $\dot x = \frac{dx}{dt}$ and $\dot y = \frac{dy}{dt}$. We then can use $\mathbf s(t)$ and $\mathbf c(t)$ to compute the state $\mathbf s( t + \Delta t)$ at a slightly later time. 

### Question 2

Use the formulas 
\begin{align*}
\mathbf v(t) &= \mathbf v(a) + \int_a^t \mathbf a(\tau)\,d\tau \\ 
\mathbf r(t) &= \mathbf r(a) + \int_a^t \mathbf v(\tau)\,d\tau \\ 
\end{align*}
to write a function `evolve` that takes:
  - `dt`: a time interval $\Delta t$
  - `s`: a state vector $\mathbf s$ (a 5-vector as described above) 
  - `c`: a control vector $\mathbf c$ (a 2-vector as above)

and returns a new state vector `dt` seconds later.

<div class="alert alert-block alert-info shadow">
    <strong>Tips:</strong> <ul> <li>Assume a <b>constant</b> acceleration throughout the interval based on $\mathbf s$ and $c_1$.</li>
    <li>Update the angle (if $c_2 \neq 0$) <b>after</b> calculating the new position and velocity. </li>
    <li>Don't forget the controls adjust the angle of the ship $2\pi$ radians per second.</li></ul>
    
If the ship is turning, getting the exact motion is more involved, but we are taking a small $\Delta t$, so we can use a simpler approximation.  
</div>

In [ ]:
def evolve(dt, s, c):
    """Returns a state vector of the ship dt seconds later assuming controls c."""
    
    # This unpacks the components of the vectors into individual variables
    x, y, vx, vy, a = s
    thrust, lr = c
    ux, uy = u_alpha(a)


    # Change the variables to their new values. 
    ...
    
    return np.array((x, y, vx, vy, a))

In [ ]:
grader.check("q2")

## Speed run

Our ship, starting from rest at the coordinates $(0,0)$, needs to get to the Green Zone without crashing into the walls.

![Initial position graphic of ship and cave](cave.png)

<!-- BEGIN QUESTION -->

### Question 3

Make a control function $\mathbf c(t)$ to navigate the course in as little time as possible. 

The input $t$ is the time, in seconds, starting from 0. Must return a 2-vector $ \langle c_1, c_2 \rangle $ where $c_1$ is 0/1 for thrust on/off and $c_2$ for -1/0/1 for left/center/right. 

<div class="alert alert-block alert-info shadow" markdown="1">
<strong>Conditionals.</strong> You can write a piecewise define function in Python by using <em>conditionals</em>. For example. the function $$f(t) = \begin{cases} t^2,  & t \leq 1 \\ 2t-1, & 1 < t <=2  \\ 3, & t> 2 \end{cases} $$ can be defined as:
<pre>
def f(t):
    if t <= 1:
        return t**2
    elif 1 < t <= 2:
        return 2*t - 1
    else:
        return 3
</pre>
</div>

In [ ]:
def c(t):
    """Return the control vector for the ship at time t."""
    
    ...
    
    # Variable names are only suggestions.
    return np.array((thrust, leftright))

<!-- END QUESTION -->

Use the code below to test your solution. Trial-and-error is a fine approach. Change the `DURATION` parameter if you need a longer or shorter window

In [ ]:
DURATION = 16
anim = make_run(DURATION, c, evolve)
HTML(anim.to_html5_video())

## Feedback

_Please use this space for any notes parts of this lab you found particularly helpful/unhelpful._

## Submission

Use the cell below to create a zip file to be uploaded to Gradescope. 

<div class="alert alert-box alert-warning shadow">
    <strong>Warning:</strong> Be sure to save the file <strong>before</strong> executing this cell, or your latest work will not be included.
</div>

In [ ]:
import zipfile, os

ASSIGNMENT_NAME="lab1"
ZIPFILE=f"{ASSIGNMENT_NAME}_submission.zip"
CWD=os.path.curdir

anim.save("speed_run.gif")

zip = zipfile.ZipFile(os.path.join(CWD, ZIPFILE), mode='w')

for filename in ["speed_run.gif", f"{ASSIGNMENT_NAME}.ipynb"]:
    zip.write(os.path.join(CWD, filename))
    
zip.close()

display(HTML(f"""Download the <a href="{os.path.join(CWD,ZIPFILE)}" download="{ZIPFILE}" target="_blank">zip file</a> and upload it to the assignment on Gradescope."""))

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()